In [1]:
import pandas as pd
import pathlib
import xarray as xr
import anndata

## Cell Metadata

In [2]:
cell_tidy_data = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
           )

it_groups = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/ITSpatial/GroupDesign/ITSpatial.cell_group_design.dissection_region_level.csv', 
    index_col=0)

cell_tidy_data = cell_tidy_data.loc[it_groups.index].copy()
cell_tidy_data['ITSpatialGroup'] = it_groups['ITSpatial']

cell_tidy_data.to_hdf('../ITSpatial.cell_tidy_data.hdf', key='data')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
cell_tidy_data.shape

(24359, 55)

## Gene rate

In [4]:
mcds_dir = '/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/'
mcds_paths = list(pathlib.Path(mcds_dir).glob('GeneWithSlop2kb.gene_da_rate.*.mcds'))

total_mcds = []
for path in mcds_paths:
    print(path)
    mcds = xr.open_dataset(path)
    cells = mcds.get_index('cell')
    cells = cells[cells.isin(cell_tidy_data.index)].copy()
    if cells.size == 0:
        continue
    use_mcds = mcds.sel(cell=cells, mc_type='CHN')['gene_da']
    total_mcds.append(use_mcds.load())

total_mcds = xr.concat(total_mcds, dim='cell')

/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.0.mcds
/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.1.mcds
/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.2.mcds
/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.3.mcds
/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.4.mcds
/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.5.mcds
/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.6.mcds
/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/GeneWithSlop2kb.gene_da_rate.7.mcds
/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/

In [5]:
total_mcds

<xarray.DataArray 'gene_da' (cell: 24359, gene: 55487)>
array([[0.71636969, 2.74982209, 0.55850604, ..., 0.14139664, 0.17969732,
        0.17969732],
       [0.26245886, 1.16932623, 0.45397878, ..., 0.41768154, 0.59690788,
        0.62965756],
       [0.23286179, 1.        , 0.58423094, ..., 0.14364194, 0.19555073,
        0.19810757],
       ...,
       [0.84476469, 1.52223891, 1.00513686, ..., 0.32540423, 1.        ,
        1.        ],
       [1.        , 1.575731  , 0.57130613, ..., 6.2992852 , 0.46824839,
        0.46824839],
       [0.63376332, 1.53429784, 0.68541633, ..., 1.        , 1.        ,
        1.        ]])
Coordinates:
    geneslop2k_chrom  (gene) object 'chr1' 'chr1' 'chr1' ... 'chrM' 'chrM'
    mc_type           <U3 'CHN'
    strand_type       <U4 'both'
    geneslop2k_end    (gene) int64 3076321 3104124 3673497 ... 16299 16299 16299
    geneslop2k_start  (gene) int64 3071252 3100015 3203900 ... 12144 13288 13355
  * gene              (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
  * cell              (cell) object '1A_M_0' '1A_M_1' ... '8B_M_997' '8B_M_999'

In [6]:
total_mcds.to_netcdf('ITSpatial.GeneWithSlop2kb.gene_da_rate.nc')

## Group level rate

In [4]:
total_mcds = xr.open_dataarray('ITSpatial.GeneWithSlop2kb.gene_da_rate.nc')
total_rate = total_mcds.to_pandas()

In [9]:
group_center = total_rate.groupby(cell_tidy_data['ITSpatialGroup']).median()

In [10]:
group_center.to_hdf('ITSpatial.GeneWithSlop2kb.gene_da_rate.group_center.hdf', key='data')

In [14]:
use_genes = pd.read_csv('TotalGeneID.txt', header=None, index_col=0).index